In [ ]:
#Como avaliar seu modelo de classificação

In [2]:
import numpy as np
import pandas as pd
import math
import sklearn
import matplotlib.pyplot as plt
import seaborn as sn


In [ ]:
from sklearn.model_selection import train_test_split
X = df_model.drop('num', axis = 1)
y = df_model['num']
SEED = 42
np.random.seed(SEED)
X_train, X_test, y_train, y_test = train_test_split(
                                   X, y, test_size = 0.20)
print('O dataset de treino possui {} pacientes e o de 
       treino {} pacientes.'
       .format(X_train.shape[0], X_test.shape[0]))

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
np.random.seed(SEED)
#avaliar o melhor valor de max_depth
model = DecisionTreeClassifier(max_depth=2)
model.fit(X_train, y_train)
predict = model.predict(X_test)
accuracy = accuracy_score(y_test, predict) * 100
print ("A acurácia foi de {:.2f}%.".format(accuracy))


In [ ]:
#usar dados novos para fazer previsões novas para o modelo.
X_news = df_aux.drop('num', axis = 1)
y_news = df_aux['num']
predict = model.predict(X_news)
accuracy = accuracy_score(y_news, predict) * 100
print ("A acurácia foi de {:.2f}%.".format(accuracy))

In [ ]:
#usando o método da validação cruzada
from sklearn.model_selection import cross_val_score
SEED = 42
np.random.seed(SEED)
model = DecisionTreeClassifier(max_depth=3)
results = cross_val_score(model, X_train, 
                          y_train, cv = 5, scoring = 'accuracy')
def intervalo(results):
    mean = results.mean()
    dv = results.std()
    print('Acurácia média: {:.2f}%'.format(mean*100))
    print('Intervalo de acurácia: [{:.2f}% ~ {:.2f}%]'
           .format((mean - 2*dv)*100, (mean + 2*dv)*100))
intervalo(results)

In [ ]:
#faz avaliação estratificada
from sklearn.model_selection import StratifiedKFold
np.random.seed(SEED)
cv = StratifiedKFold(n_splits = 5, shuffle = True)
model = DecisionTreeClassifier(max_depth=3)
results = cross_val_score(model, X_train, y_train, cv = cv)
intervalo(results)

In [ ]:
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
np.random.seed(SEED)
cv = StratifiedKFold(n_splits = 5, shuffle = True)
model = DecisionTreeClassifier(max_depth=3)
model_svc = SVC()
model_log = LogisticRegression(solver='liblinear')
model_rand = RandomForestClassifier(n_estimators=100)
models = [model, model_svc, model_log, model_rand]
name = ['Árvore de Decisão', 'SVC', 
        'Regressão Logística', 'Random Forest']
count = 0
for item in models:
    np.random.seed(SEED)
    results = cross_val_score(item, X_train, y_train, cv = cv, 
                              scoring = 'accuracy')
    mean = results.mean()
    dv = results.std()
    print('Acurácia média - Modelo {}: {:.2f}%'
          .format(name[count], mean*100))
    print('Intervalo de acurácia - 
           Modelo {}: [{:.2f}% ~ {:.2f}%]\n'
           .format(name[count], 
                  (mean - 2*dv)*100, (mean + 2*dv)*100))
    count += 1

In [ ]:
#precision
def intervalo_prec(results):
    mean = results.mean()
    dv = results.std()
    print('Precisão média: {:.2f}%'.format(mean*100))
    print('Intervalo de Precisão: [{:.2f}% ~ {:.2f}%]'
          .format((mean - 2*dv)*100, (mean + 2*dv)*100))
np.random.seed(SEED)
cv = StratifiedKFold(n_splits = 5, shuffle = True)
model = LogisticRegression(solver='liblinear')
results = cross_val_score(model, X_train, y_train, cv = cv, 
                          scoring = 'precision')
intervalo_prec(results)

In [ ]:
#recall
def intervalo_recall(results):
    mean = results.mean()
    dv = results.std()
    print('Recall médio: {:.2f}%'.format(mean*100))
    print('Intervalo de Recall: [{:.2f}% ~ {:.2f}%]'
          .format((mean - 2*dv)*100, (mean + 2*dv)*100))
np.random.seed(SEED)
cv = StratifiedKFold(n_splits = 5, shuffle = True)
model = LogisticRegression(solver='liblinear')
results = cross_val_score(model, X_train, y_train, cv = cv, scoring = 'recall')
intervalo_recall(results)

In [ ]:
#fazendo o precision e o racall
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
np.random.seed(SEED)
cv = StratifiedKFold(n_splits = 5, shuffle = True)
model = DecisionTreeClassifier(max_depth=3)
model_svc = SVC()
model_log = LogisticRegression(solver='liblinear')
model_rand = RandomForestClassifier(n_estimators=100)
models = [model, model_svc, model_log, model_rand]
name = ['Árvore de Decisão', 'SVC', 
        'Regressão Logística', 'Random Forest']
count = 0
for item in models:
    np.random.seed(SEED)
    results = cross_val_score(item, X_train, y_train, cv = cv, 
                              scoring = 'accuracy')
    mean = results.mean()
    dv = results.std()
    print('Acurácia média - Modelo {}: {:.2f}%'
          .format(name[count], mean*100))
    print('Intervalo de acurácia - 
           Modelo {}: [{:.2f}% ~ {:.2f}%]\n'
           .format(name[count], 
                  (mean - 2*dv)*100, (mean + 2*dv)*100))
    count += 1

In [ ]:
#fazendo a matriz de confusão
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
np.random.seed(SEED)
cv = StratifiedKFold(n_splits = 5, shuffle = True)
model = LogisticRegression(solver='liblinear')
y_pred = cross_val_predict(model, X_train, y_train, cv = cv)
fig, ax = plt.subplots()
sns.heatmap(confusion_matrix(y_train, y_pred), annot=True, 
            ax=ax, fmt='d', cmap='Reds')
ax.set_title("Matriz de Confusão", fontsize=18)
ax.set_ylabel("True label")
ax.set_xlabel("Predicted Label")
plt.tight_layout()

In [ ]:
#relatório do modelo
from sklearn.metrics import classification_report
# relatório do modelo
print('Relatório de classificação:\n', classification_report(y_train, y_pred, digits=4))

In [ ]:
#curva das características operacionais do receptor (Receiver operating characteristic — ROC) 
from sklearn.metrics import roc_curve
fpr, tpr, thresholds = roc_curve(y_train, y_scores)
fig, ax = plt.subplots(figsize = (12,4))
plt.plot(fpr, tpr, linewidth=2, label = 'Logistic Regression')
plt.plot([0,1], [0,1], 'k--')
plt.axis([0, 1, 0, 1])
plt.xlabel('Taxa de Falsos Positivos')
plt.ylabel('Taxa de Verdadeiros Positivos')
plt.legend(loc = 'lower right')
plt.title('Curva ROC', fontsize = 14)
plt.show()

In [ ]:
from sklearn.metrics import roc_auc_score
print('Área sob a curva ROC: {:.4f}'
      .format(roc_auc_score(y_train, y_scores)))

In [ ]:
#fazendo o teste
np.random.seed(SEED)
final_model = LogisticRegression(solver='liblinear')
final_model.fit(X_train, y_train)
y_pred = final_model.predict(X_test)
y_prob = final_model.predict_proba(X_test)
# imprimir relatório de classificação
print("Relatório de Classificação:\n", 
       classification_report(y_test, y_pred, digits=4))
# imprimir a área sob a curva
print("AUC: {:.4f}\n".format(roc_auc_score(y_test,y_prob[:,1])))

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier 
from sklearn.linear_model import LogisticRegression 
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score
cv = StratifiedKFold(n_splits = 5, shuffle = True)
model1 = LogisticRegression()
model2 = DecisionTreeClassifier(max_depth=3) 
model3 = RandomForestClassifier(n_estimators=100)
results = cross_val_score(model3, x_train, y_train, cv = cv)
print(results)

In [ ]:
#FIM